# Segmenting and Clustering Neighborhoods in Toronto

### 0. Importing necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [2]:
# Library for Wikipedia scraping
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
beautifulsoup4            4.6.3                    py35_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
lxml                      4.2.5            py35hc9114bc_0    conda-forge


In [3]:
from bs4 import BeautifulSoup

### 1. Download and Explore Dataset

First we look at the needed page to find the place, where the needed table is

In [4]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_page=requests.get(wikipedia_link)
Toronto_page=raw_page.text
soup=BeautifulSoup(Toronto_page,'lxml')
print (soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":867606113,"wgRevisionId":867606113,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

By looking through the html text we find the table under `table class="wikitable sortable"`. So let's look at this part more closely

In [5]:
Neib_table = soup.find('table',{'class':'wikitable sortable'})
Neib_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Each line of the table is in `tr` tag, each field is in `td` tag. We loop through the lines and fields and read data into 3 lists

In [6]:
trs=Neib_table.find_all('tr')
PostalCode_list=[]
Borough_list=[]
Neigh_list=[]
for tr in trs:
    temp_list=[]
    for child in tr.find_all('td'):
        temp_list.append(child.contents[0].string)
    try:
        PostalCode_list.append(temp_list[0])
        Borough_list.append(temp_list[1])
        Neigh_list.append(temp_list[2])
    except:
        continue

We create a data frame and fill it with the data from our 3 lists

In [7]:
TorDF=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
TorDF['PostalCode']=PostalCode_list
TorDF['Borough']=Borough_list
TorDF['Neighborhood']=Neigh_list
TorDF.head(15)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [8]:
TorDF.shape

(289, 3)

**Now we have to process the data in our data frame:**

1. We delete the lines where Borough = "not assigned"

In [9]:
TorDF=TorDF[TorDF['Borough'].map(lambda x:x.find('Not assigned')==-1)]
TorDF=TorDF.reset_index(drop=True)
TorDF.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [10]:
TorDF.shape

(212, 3)

2. Next we update neighborhoods with 'not assigned' names to Borough names

In [11]:
TorDF['Neighborhood']=np.where(TorDF['Neighborhood'].map(lambda x:x.find('Not assigned')==-1),TorDF['Neighborhood'],TorDF['Borough'])
TorDF.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


3. Finally we list neiborhoods which belong to the same PostalCode+Borough in one line of the table

In [12]:
resTorDF=pd.DataFrame(columns=['PostalCode','Borough','Neighborhoods'])
groupedDF=TorDF.groupby(['PostalCode','Borough'])
for (k1,k2),group in groupedDF:
    temp_str=", ".join(group['Neighborhood'].tolist())
    temp_str=temp_str.replace('\n','')
    resTorDF=resTorDF.append({'PostalCode':k1,'Borough':k2,'Neighborhoods':temp_str},ignore_index=True)

resTorDF.head(50)

,PostalCode,Borough,Neighborhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**Shape of the resulting table**

In [13]:
resTorDF.shape

(103, 3)

Next step is to find the coordinates for each Postal code

In [14]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
TorPC_DF=pd.merge(resTorDF,CoordDF,how='left',on='PostalCode')
print(TorPC_DF.shape)
TorPC_DF.head()

(103, 5)


,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 2. Explore and Prepare Data about Neighborhoods

The analysis will be performed for PostalCodes, as the neighborhoods in Toronto seem to be very small so the results will intersect a lot. The following functions are changed accordingly

In [16]:
CLIENT_ID = '***' # Foursquare ID
CLIENT_SECRET = '***' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

Create function to get nearby venues for given coordinates

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        LIMIT=100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
                
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's choose a single largest borough in Toronto and perform the analysis on it

In [18]:
TorPC_DF.groupby('Borough').count()

,PostalCode,Neighborhoods,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [19]:
TorChosen=TorPC_DF[TorPC_DF['Borough']=='North York']
#TorChosen=TorPC_DF
TorChosen=TorChosen.reset_index(drop='True')
TorChosen#.shape

,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [20]:
Tor_venues = getNearbyVenues(names=TorChosen['PostalCode'],
                                   latitudes=TorChosen['Latitude'],
                                   longitudes=TorChosen['Longitude']
                                  )

M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M5M
M6A
M6B
M6L
M9L
M9M


In [21]:
print('There are {} unique categories.'.format(len(Tor_venues['Venue Category'].unique())))

There are 104 unique categories.


In [22]:
# one hot encoding
Tor_onehot = pd.get_dummies(Tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tor_onehot['PostalCode'] = Tor_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [Tor_onehot.columns[-1]] + list(Tor_onehot.columns[:-1])
Tor_onehot = Tor_onehot[fixed_columns]

Tor_onehot.head()

,PostalCode,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,M2H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M2H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M2H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M2H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M2J,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
Tor_grouped = Tor_onehot.groupby('PostalCode').mean().reset_index()
Tor_grouped

,PostalCode,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,M2H,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.25,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M2J,0.000000,0.000000,0.015385,0.000000,0.030769,0.0,0.0,0.046154,0.030769,0.0,0.015385,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.015385,0.015385,0.030769,0.000000,0.00,0.0,0.000000,0.015385,0.000000,0.00,0.138462,0.061538,0.00,0.00,0.015385,0.015385,0.015385,0.015385,0.00,0.000000,0.000000,0.00,0.0,0.015385,0.0,0.000000,0.076923,0.0,0.030769,0.0,0.00,0.000000,0.015385,0.000000,0.00,0.015385,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.015385,0.00,0.015385,0.000000,0.015385,0.0,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.046154,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.000000,0.015385,0.000000,0.015385,0.00,0.015385,0.030769,0.015385,0.000000,0.000000,0.015385,0.015385
2,M2K,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.25,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.250000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M2L,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.

In [24]:
Tor_grouped.shape

(23, 105)

The number of PCs in Tor_grouped is 23, but in TorChosen there were 24 PCs. Let's find missing PC

In [25]:
testDF=pd.merge(TorChosen,Tor_grouped,on='PostalCode',how='left')
testDF
#TorChosenCleared=TorChosen[TorChosen['PostalCode']==testDF[testDF['Airport']=='NaN']]
#TorChosenCleared

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.25,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.000000,0.000000,0.015385,0.000000,0.030769,0.0,0.0,0.046154,0.030769,0.0,0.015385,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.015385,0.015385,0.030769,0.000000,0.00,0.0,0.000000,0.015385,0.000000,0.00,0.138462,0.061538,0.00,0.00,0.015385,0.015385,0.015385,0.015385,0.00,0.000000,0.000000,0.00,0.0,0.015385,0.0,0.000000,0.076923,0.0,0.030769,0.0,0.00,0.000000,0.015385,0.000000,0.00,0.015385,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.015385,0.00,0.015385,0.000000,0.015385,0.0,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.046154,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.000000,0.015385,0.000000,0.015385,0.00,0.015385,0.030769,0.015385,0.000000,0.000000,0.015385,0.015385
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.25,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.250000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [26]:
TorChosenCleared=TorChosen[TorChosen['PostalCode']!='M2M']
TorChosenCleared=TorChosenCleared.reset_index(drop=True)
TorChosenCleared.shape

(23, 5)

Now it's time to choose most common venues for each PostalCode. The information will be then used for clustering

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
PC_venues_sorted = pd.DataFrame(columns=columns)
PC_venues_sorted['PostalCode'] = Tor_grouped['PostalCode']

for ind in np.arange(Tor_grouped.shape[0]):
    PC_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tor_grouped.iloc[ind, :], num_top_venues)

PC_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M2J,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Restaurant,Food Court,Toy / Game Store,Bank,Asian Restaurant,Bus Station
2,M2K,Café,Bank,Chinese Restaurant,Japanese Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
3,M2L,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2N,Restaurant,Coffee Shop,Ramen Restaurant,Sandwich Place,Pizza Place,Fast Food Restaurant,Café,Movie Theater,Pet Store,Pharmacy
5,M2P,Park,Bank,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
6,M2R,Pharmacy,Pizza Place,Coffee Shop,Grocery Store,Butcher,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
7,M3A,Park,Food & Drink Shop,BBQ Joint,Fast Food Restaurant,Bus Stop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
8,M3B,Japanese Restaurant,Caribbean Restaurant,Café,Pool,Gym / Fitness Center,Baseball Field,Basketball Court,Women's Store,Comfort Food Restaurant,Construction & Landscaping
9,M3C,Coffee Shop,Asian Restaurant,Gym,Beer Store,General Entertainment,Sporting Goods Shop,Japanese Restaurant,Italian Restaurant,Dim Sum Restaurant,Restaurant


### 3. Cluster Neighborhoods

In [29]:
# set number of clusters
kclusters = 5

Tor_grouped_clustering = Tor_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)#[0:10] 

23

In [30]:
Tor_merged = TorChosenCleared

# add clustering labels
Tor_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Tor_merged = Tor_merged.join(PC_venues_sorted.set_index('PostalCode'), on='PostalCode')

Tor_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Restaurant,Food Court,Toy / Game Store,Bank,Asian Restaurant,Bus Station
2,M2K,North York,Bayview Village,43.786947,-79.385975,0,Café,Bank,Chinese Restaurant,Japanese Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,3,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2N,North York,Willowdale South,43.770120,-79.408493,0,Restaurant,Coffee Shop,Ramen Restaurant,Sandwich Place,Pizza Place,Fast Food Restaurant,Café,Movie Theater,Pet Store,Pharmacy


**The following map shows clustered Postal codes within North York borough. Each of 5 clusters has it's own color**

In [31]:
latitude=43.7615
longitude=-79.4111

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tor_merged['Latitude'], Tor_merged['Longitude'], Tor_merged['PostalCode'], Tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Next we list the postal codes in each cluster in order to think about common features of each cluster.**

In [32]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 0, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Restaurant,Food Court,Toy / Game Store,Bank,Asian Restaurant,Bus Station
2,North York,0,Café,Bank,Chinese Restaurant,Japanese Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
4,North York,0,Restaurant,Coffee Shop,Ramen Restaurant,Sandwich Place,Pizza Place,Fast Food Restaurant,Café,Movie Theater,Pet Store,Pharmacy
6,North York,0,Pharmacy,Pizza Place,Coffee Shop,Grocery Store,Butcher,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
8,North York,0,Japanese Restaurant,Caribbean Restaurant,Café,Pool,Gym / Fitness Center,Baseball Field,Basketball Court,Women's Store,Comfort Food Restaurant,Construction & Landscaping
9,North York,0,Coffee Shop,Asian Restaurant,Gym,Beer Store,General Entertainment,Sporting Goods Shop,Japanese Restaurant,Italian Restaurant,Dim Sum Restaurant,Restaurant
10,North York,0,Coffee Shop,Bank,Pharmacy,Deli / Bodega,Restaurant,Diner,Sandwich Place,Pizza Place,Shopping Mall,Bridal Shop
11,North York,0,Furniture / Home Store,Coffee Shop,Massage Studio,Bar,Eastern European Restaurant,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
13,North York,0,Grocery Store,Shopping Mall,Bank,Hotel,Women's Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


In [33]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 1, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,1,Baseball Field,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [34]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 2, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,2,Food Truck,Baseball Field,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


In [35]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 3, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,3,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [36]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 4, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,North York,4,Park,Bank,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
7,North York,4,Park,Food & Drink Shop,BBQ Joint,Fast Food Restaurant,Bus Stop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
12,North York,4,Park,Airport,Bus Stop,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
